### load lib

In [1]:
from datasets import load_dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from simplet5 import SimpleT5
from tqdm import tqdm
from rouge_score import rouge_scorer

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
Global seed set to 42


### load dataset

In [2]:
raw_datasets = load_dataset("squad_v1_pt")

Using custom data configuration default
Reusing dataset squad_v1_pt (/home/studio-lab-user/.cache/huggingface/datasets/squad_v1_pt/default/1.1.0/65162e0fbe44f19a4d2ad9f5f507d2e965e74249fc3239dc78b4e3bd93bab7c4)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
train = raw_datasets['train'].to_pandas()
validation =  raw_datasets['validation'].to_pandas()

### create variavel resposta

In [4]:
lista_train = []
for i in range(len(train)):
  lista_train.append(train['answers'][i]['text'][0])
train['resp'] = lista_train

lista_val = []
for i in range(len(validation)):
  lista_val.append(train['answers'][i]['text'][0])
validation['resp'] = lista_val

### create dataset train

In [5]:
q_list = "answers: " + train['resp']                     # answers  list to feed the model
n_list = train['question'] + train['context'] + " </s>"  # question list to feed the model

dict_data = {'source_text': n_list,
      'target_text': q_list}

train = pd.DataFrame(dict_data)
train.head()  

,source_text,target_text
0,A quem a Virgem Maria supostamente apareceu em...,answers: Saint Bernadette Soubirous
1,O que fica em frente ao edifício principal de ...,answers: uma estátua de cobre de Cristo
2,A Basílica do Sagrado Coração em Notre Dame fi...,answers: o edifício principal
3,O que é a gruta de Notre Dame?Arquitetonicamen...,answers: um lugar mariano de oração e reflexão
4,O que fica no topo do edifício principal em No...,answers: uma estátua de ouro da Virgem Maria


### create dataset test

In [6]:
q_list = "answers: " + validation['resp']                     # answers  list to feed the model
n_list = validation['question'] + validation['context'] + " </s>"  # question list to feed the model

dict_data = {'source_text': n_list,
      'target_text': q_list}

validation = pd.DataFrame(dict_data)
validation.head()  

,source_text,target_text
0,Qual time da NFL representou o AFC no Super Bo...,answers: Saint Bernadette Soubirous
1,Qual time da NFL representou o NFC no Super Bo...,answers: uma estátua de cobre de Cristo
2,Onde aconteceu o Super Bowl 50?Super Bowl 50 f...,answers: o edifício principal
3,Qual time da NFL venceu o Super Bowl 50?Super ...,answers: um lugar mariano de oração e reflexão
4,Que cor foi usada para enfatizar o 50º anivers...,answers: uma estátua de ouro da Virgem Maria


In [7]:
train = train[0:5000]
train.shape

(5000, 2)

In [8]:
train.shape, len(train.source_text.unique()), len(train.target_text.unique())

((5000, 2), 4969, 3615)

In [9]:
train_data, val_data = train_test_split(train, test_size=0.2)
test_data = validation[:1000]
train_data.shape, val_data.shape, test_data.shape

((4000, 2), (1000, 2), (1000, 2))

### train model

In [10]:
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="unicamp-dl/ptt5-base-portuguese-vocab")
model.train(train_df = train_data,
            eval_df = val_data, 
            source_max_token_len=128, 
            target_max_token_len=50, 
            batch_size=16, max_epochs=3, use_gpu=True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: /home/studio-lab-user/lightning_logs

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 42
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

### inferencia

In [11]:
#requer carregar o modelo para gerar inferencia
model.load_model("t5","outputs/simplet5-epoch-2-train-loss-0.9372-val-loss-1.0509", use_gpu=True)

In [37]:
contexto = context = r"""
A pandemia de COVID-19, também conhecida como pandemia de coronavírus, é uma pandemia em curso de COVID-19, 
uma doença respiratória aguda causada pelo coronavírus da síndrome respiratória aguda grave 2 (SARS-CoV-2). 
A doença foi identificada pela primeira vez em Wuhan, na província de Hubei, República Popular da China, 
em 1 de dezembro de 2019, mas o primeiro caso foi reportado em 31 de dezembro do mesmo ano. 
Acredita-se que o vírus tenha uma origem zoonótica, porque os primeiros casos confirmados 
tinham principalmente ligações ao Mercado Atacadista de Frutos do Mar de Huanan, que também vendia animais vivos. 
Em 11 de março de 2020, a Organização Mundial da Saúde declarou o surto uma pandemia. Até 8 de fevereiro de 2021, 
pelo menos 105 743 102 casos da doença foram confirmados em pelo menos 191 países e territórios, 
com cerca de 2 308 943 mortes e 58 851 440 pessoas curadas.
"""

In [38]:
question1 = "Quando começou a pandemia de Covid-19 no mundo?"
question2 = "Qual é a data de início da pandemia Covid-19 em todo o mundo?"
question3 = "O qué a pandemia de Covid-19?"

In [39]:
q1 = question1 + context
q2 = question2 + context
q3 = question3 + context

In [40]:
#question 1 
model.predict(q1)

['answers: 21 de fevereiro de 2021']

In [41]:
#question 2
model.predict(q2)

['answers: 8 de fevereiro de 2021']

In [42]:
#question 3
model.predict(q3)

['answers: doença respiratória aguda causada pelo coronavírus da síndrome respiratória aguda grave 2']

### Y_pred / Y_test

In [43]:
y_test = [a for a in test_data['target_text']]

# get the predicted answers
y_pred = []
for q in tqdm(test_data['source_text']):
    y_pred.append(model.predict(q)[0])


100%|██████████| 1000/1000 [02:32<00:00,  6.55it/s]


### Score

In [45]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

In [46]:
metricas = []
for i in range(len(y_test)):
  metricas.append(scorer.score(y_pred[i], y_test[i]))

precision = []
for i in range(len(y_test)):
  precision.append(metricas[i]['rougeL'][0])  

recall = []
for i in range(len(y_test)):
  recall.append(metricas[i]['rougeL'][1])  

In [47]:
recall = np.mean(recall)
precision = np.mean(precision)

### F1

In [51]:
f1_score = 2 * (precision * recall) / (precision + recall)

In [52]:
f1_score

0.3382399704764368